## ATIVIDADE 1 - Web Scraping em Ambiente Controlado

Importações de **bibliotecas** utilizadas

In [1]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import time
import csv
import os
import ast

Definindo as **variáveis globais** que serão utilizadas ao longo do código

In [2]:
baseUrl = 'http://127.0.0.1:8000/places/default'

ids = ['places_country__row', 'places_currency_name__row', 'places_continent__row', 'places_neighbours__row']
pages = set()
info = []

## Funções

Função para pegar os links de páginas dos **países** (a partir da página *index*)

In [3]:
def getLinks(pageUrl):
    global pages, baseUrl
    fullUrl = urljoin(baseUrl, pageUrl)

    try:
        html = urlopen(fullUrl)
        bs = BeautifulSoup(html, 'html.parser')

        for link in bs.find_all('a'):
            href = link.get('href')
            newPage = urljoin(baseUrl, href)
            if newPage not in pages and 'user' not in href:
                if 'index' in href or 'view' in href:
                    pages.add(newPage)
                    getLinks(href)
    except HTTPError as e:
        print(f'Erro HTTP [{e.code}] tentando acessar [{fullUrl}]')
    except Exception as e:
        print(f'Deu pau :( [{e}]')

Função **pega_nome** trabalha juntamente com a **procura_info**, onde a última pega todas as informações necessárias das páginas - *Nome, Continente, Moeda* - e a primeira, além de entrar nos links dos países vizinho, pega o nome deles

In [4]:
def pega_nome(classe):
    aux = []
    for vizinho in classe.find_all('a'):
        url = urljoin(baseUrl, vizinho['href'])
        html = urlopen(url)
        bs = BeautifulSoup(html, 'html.parser')
        
        if 'iso' in url:
            linha = bs.find(id='places_country__row')
            if linha:
                nome = linha.find(class_='w2p_fw')
                if nome:
                    aux.append(nome.text.strip())
            
    return aux        

In [5]:
def procura_info(aux, bs):
    global ids, info
    
    for i in ids:
        linha = bs.find(id=i)
        classe = linha.find(class_='w2p_fw')

        if i == 'places_neighbours__row':
            aux.append(pega_nome(classe))
        else:
            aux.append(classe.text.strip())

    aux.append(time.ctime())
    info.append(aux)

Salva as informações presentes no array em um **.csv**

In [6]:
def salva_csv():
    global info
    colunas = ['País', 'Moeda', 'Continente', 'Países Vizinhos', 'Horário']

    with open('Paises.csv', mode='w', newline='') as arq:
        w = csv.writer(arq)
        w.writerow(colunas)
        w.writerows(info)

print('Deu certo')

Deu certo


In [7]:
def carrega_csv():
    paises_csv = {}
    if os.path.exists('Paises.csv'):
        with open('Paises.csv', mode='r', newline='') as arq:
            r = csv.reader(arq)
            next(r)
            for linha in r:
                pais = linha[0]
                paises_csv[pais] = linha
    return paises_csv

In [8]:
def salva_info_csv(nova, antiga):            
    for pais in nova:
        nome = pais[0]
        
        if isinstance(antiga[nome][3], str):
            antiga[nome] = list(antiga[nome])
            antiga[nome][3] = ast.literal_eval(antiga[nome][3])
            
        pais_antigo = antiga[nome]
        
        if pais[:-1] != pais_antigo[:-1]:
            antiga[pais[0]] = pais 
    
    with open('Paises.csv', mode='w', newline='') as arq:
        w = csv.writer(arq)
        w.writerow(['País', 'Moeda', 'Continente', 'Países Vizinhos', 'Horário'])
        for linha in antiga.values():
            w.writerow(linha)


## Main

Partes de código em que as funções são chamadas e testes são realizados

In [9]:
primeira_vez = False

while True:
    print(' -- [ INICIANDO ITERAÇÃO ] --')
    
    print(' -- [ PEGANDO LINKS ] --')
    getLinks('/places/default/index')

    print(' -- [ PEGANDO INFORMAÇÕES ] --')
    for page in pages:
        if 'index' not in page:
            aux = []
            html = urlopen(page)
            bs = BeautifulSoup(html, 'html.parser')

            procura_info(aux, bs)
    
    if primeira_vez: 
        print(' -- [ SALVANDO INFORMAÇÕES NO CSV (1° VEZ) ] --')
        salva_csv()
        primeira_vez = False
    else:
        print('-- [ SALVANDO INFORMAÇÕES ] --')
        antiga = carrega_csv()
        salva_info_csv(info, antiga)
        break
        
    print(' -- [ DORMINDO POR 60S ] --')
    time.sleep(60)
                

 -- [ INICIANDO ITERAÇÃO ] --
 -- [ PEGANDO LINKS ] --
 -- [ PEGANDO INFORMAÇÕES ] --
-- [ SALVANDO INFORMAÇÕES ] --
pais[:-1]:  ['Norfolk Island', 'Dollar', 'OC', []]
pais_antigo[:-1]:  ['Norfolk Island', 'Dollar', 'OC', []]
pais[:-1]:  ['Serbia and Montenegro', 'Dinar', 'EU', ['Albania', 'Hungary', 'Macedonia', 'Romania', 'Croatia', 'Bosnia and Herzegovina', 'Bulgaria']]
pais_antigo[:-1]:  ['Serbia and Montenegro', 'Dinar', 'EU', ['Albania', 'Hungary', 'Macedonia', 'Romania', 'Croatia', 'Bosnia and Herzegovina', 'Bulgaria']]
pais[:-1]:  ['Turkey', 'Lira', 'AS', ['Syria', 'Georgia', 'Iraq', 'Iran', 'Greece', 'Armenia', 'Azerbaijan', 'Bulgaria']]
pais_antigo[:-1]:  ['Turkey', 'Lira', 'AS', ['Syria', 'Georgia', 'Iraq', 'Iran', 'Greece', 'Armenia', 'Azerbaijan', 'Bulgaria']]
pais[:-1]:  ['Solomon Islands', 'Dollar', 'OC', []]
pais_antigo[:-1]:  ['Solomon Islands', 'Dollar', 'OC', []]
pais[:-1]:  ['Ecuador', 'Dollar', 'SA', ['Peru', 'Colombia']]
pais_antigo[:-1]:  ['Ecuador', 'Dollar', 'S